In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table-like data and matrices
import numpy as np
import pandas as pd
from pandas import DataFrame,Series
import csv
import pprint

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Others
import math
from datetime import datetime,timedelta

# Configure visualisations
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6

In [2]:
links = pd.read_csv('datas/train/links_(table_3).csv')
links= links.set_index('link_id')
routes = pd.read_csv('datas/train/routes_(table_4).csv')

# Step 1: Load trajectories and split by link id

In [3]:
trajectories = []
with open('datas/train/trajectories(table_5)_training.csv') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        travel_seq = row["travel_seq"].split(";")
        for travel in travel_seq:
            cur_row = row.copy()
            cur_row["total_travel_time"] = cur_row["travel_time"]
            link_id,enter_time,travel_time = travel.split("#")
            cur_row["link_id"] = link_id
            cur_row["enter_time"] = enter_time
            cur_row["travel_time"] = travel_time
            cur_row["travel_seq"] = ','.join(map(lambda s:s.split("#")[0],travel_seq))
            trajectories.append(cur_row)

In [4]:
pd.DataFrame(trajectories[:5])

,enter_time,intersection_id,link_id,starting_time,tollgate_id,total_travel_time,travel_seq,travel_time,vehicle_id
0,2016-07-19 00:14:24,B,105,2016-07-19 00:14:24,3,70.85,"105,100,111,103,122",9.56,1065642
1,2016-07-19 00:14:34,B,100,2016-07-19 00:14:24,3,70.85,"105,100,111,103,122",6.75,1065642
2,2016-07-19 00:14:41,B,111,2016-07-19 00:14:24,3,70.85,"105,100,111,103,122",13.00,1065642
3,2016-07-19 00:14:54,B,103,2016-07-19 00:14:24,3,70.85,"105,100,111,103,122",7.47,1065642
4,2016-07-19 00:15:02,B,122,2016-07-19 00:14:24,3,70.85,"105,100,111,103,122",32.85,1065642


# Step 2: Create a dictionary to store travel time for each link for per time window

In [5]:
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(trajectories)):
    each_traj = trajectories[i]
    link_id = each_traj['link_id']
    if link_id not in travel_times.keys():
        travel_times[link_id] = {}

    trace_start_time = each_traj['starting_time']
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                 trace_start_time.hour, time_window_minute, 0)
    tt = float(each_traj['travel_time']) # travel time

    link_id_node = travel_times[link_id]
    if start_time_window not in link_id_node.keys():
        link_id_node[start_time_window] = [tt]
    else:
        link_id_node[start_time_window].append(tt)

# Step 3: Calculate average travel time for each route per time window

In [105]:
out_lines = []
header = ['link_id','time_window', 'avg_travel_time','travel_time','flow']
for link_id in travel_times.keys():
        link_node = travel_times[link_id]
        route_time_windows = list(link_node.keys())
        for time_window_start in route_time_windows:
            time_window_end = time_window_start + timedelta(minutes=20)
            tt_set = link_node[time_window_start]
            avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
            flow = len(tt_set)
            out_line = [link_id,time_window_start,avg_tt,tt_set,flow]
            out_lines.append(out_line)

In [106]:
avg_travel_time_df = pd.DataFrame(out_lines,columns=header).sort(columns=['link_id','time_window'])
avg_travel_time_df.index = range(len(avg_travel_time_df))
avg_travel_time_df.head()

,link_id,time_window,avg_travel_time,travel_time,flow
0,100,2016-07-19 00:00:00,6.75,[6.75],1
1,100,2016-07-19 00:20:00,5.15,"[7.44, 2.85]",2
2,100,2016-07-19 00:40:00,12.34,[12.34],1
3,100,2016-07-19 01:20:00,9.35,[9.35],1
4,100,2016-07-19 01:40:00,7.68,"[6.28, 9.07]",2


# Step4: Save JSON File

In [107]:
avg_travel_time_df.to_csv('datas/train/20min_avg_travel_timetrajectories(table_5)_training.csv',index=False)

# Convert avg_travel_time to speed

In [108]:
links_dict = links.to_dict('index')

In [109]:
links_dict[100]

{'in_top': '105',
 'lane_width': 3,
 'lanes': 1,
 'length': 58,
 'out_top': '111',
 'width': 3}

In [110]:
speeds = []
for link_id in links_dict.keys():
    link_avg_travel_time = (avg_travel_time_df[avg_travel_time_df.link_id == str(link_id)])['avg_travel_time']
    speed = (links_dict[link_id]['length'] / link_avg_travel_time)
    speed.name = 'speed'
    speeds.append(round(speed, 2))
speed_series = pd.concat(speeds)
avg_travel_time_df['speed'] = speed_series

In [111]:
avg_travel_time_df.head()

,link_id,time_window,avg_travel_time,travel_time,flow,speed
0,100,2016-07-19 00:00:00,6.75,[6.75],1,8.59
1,100,2016-07-19 00:20:00,5.15,"[7.44, 2.85]",2,11.26
2,100,2016-07-19 00:40:00,12.34,[12.34],1,4.70
3,100,2016-07-19 01:20:00,9.35,[9.35],1,6.20
4,100,2016-07-19 01:40:00,7.68,"[6.28, 9.07]",2,7.55


# Describe the travel time and speed

In [112]:
avg_travel_time_df.groupby('link_id').describe().unstack()

avg_travel_time                                                       \
                  count       mean        std   min      25%    50%      75%   
link_id                                                                        
100              5180.0   7.731950   4.832119  1.40   5.8000   6.93   8.3500   
101              4247.0   9.883725   6.735573  2.03   6.8200   8.24  10.3200   
102              4051.0  10.917122   6.981095  3.28   8.9700  10.06  11.6400   
103              5506.0   7.346244   8.656201  0.55   3.0000   4.40   8.1900   
104              4048.0  28.501309  17.718740  7.33  21.2100  24.65  30.2125   
105              5182.0  12.244054   8.471746  1.88   8.7400  10.79  13.7900   
106              4248.0   2.496815   4.680724  0.36   1.3300   1.63   2.1000   
107              6225.0   4.738490   4.912639  0.83   3.3700   4.16   5.0100   
108              6225.0   5.677735   5.296193  0.97   4.2400   5.00   5.8400   
109              4049.0  10.784075   6.184843  3.38   8.6700   9.79  11.3900   
110              6225.0  12.973804   5.734569  2.65  10.4600  12.07  14.2300   
111              5506.0  29.750418  17.582302  2.92  18.0725  26.28  36.8200   
112              4025.0  28.820383  18.919255  4.98  17.6000  24.72  34.8900   
113              4253.0  25.457216  17.562017  3.48  16.3100  21.86  29.9300   
114              5300.0  22.314685  25.935645  4.81  15.7900  18.16  21.5200   
115              4051.0  10.937509   5.775604  2.68   8.6700   9.93  11.7500   
116              4247.0  18.702258  13.365417  4.35  14.3600  16.43  19.3650   
117              5965.0  33.026136  22.831953  3.60  22.0600  28.97  37.7000   
118              5300.0  31.170358  23.262630  5.00  20.1300  24.82  33.6925   
119              5300.0   1.153575   1.910469  0.22   0.7800   0.93   1.1200   
120              5965.0   1.055412   2.387878  0.16   0.7100   0.84   0.9900   
121              4248.0  18.761349  17.245908  2.85  11.0975  14.62  22.1000   
122              5839.0  39.095333  23.563154  4.75  28.9950  35.30  43.2950   
123              6225.0   6.713080   4.566265  1.43   5.2800   6.15   7.1200   

                    flow             ...                 speed             \
             max   count       mean  ...     75%   max   count       mean   
link_id                              ...                                    
100       130.46  5180.0   4.858880  ...     6.0  34.0  5180.0   8.612376   
101        92.66  4247.0   3.715328  ...     5.0  30.0  4247.0  10.221074   
102       377.43  4051.0   3.211800  ...     4.0  21.0  4051.0  12.965754   
103       125.20  5506.0   6.873774  ...     9.0  47.0  5506.0   5.793035   
104       506.95  4048.0   3.206522  ...     4.0  21.0  4048.0  11.740892   
105       270.55  5182.0   4.897144  ...     7.0  34.0  5182.0   7.573489   
106       181.18  4248.0   3.718927  ...     5.0  30.0  4248.0   9.448806   
107       220.19  6225.0  11.357590  ...    17.0  60.0  6225.0   8.558893   
108       228.00  6225.0  11.360000  ...    17.0  60.0  6225.0   8.270281   
109       174.85  4049.0   3.209928  ...     4.0  21.0  4049.0  13.745710   
110       135.39  6225.0  11.381847  ...    17.0  60.0  6225.0   9.193981   
111       232.82  5506.0   6.879041  ...     9.0  47.0  5506.0   5.458327   
112       498.15  4025.0   3.151056  ...     4.0  21.0  4025.0   8.604402   
113       318.94  4253.0   3.740183  ...     5.0  30.0  4253.0   7.557745   
114      1332.63  5300.0   4.389057  ...     6.0  29.0  5300.0  10.760564   
115       247.11  4051.0   3.212787  ...     4.0  21.0  4051.0  10.842950   
116       516.86  4247.0   3.712267  ...     5.0  30.0  4247.0  10.916157   
117       832.52  5965.0   7.953562  ...    11.0  48.0  5965.0   5.138795   
118       615.63  5300.0   4.393208  ...     6.0  31.0  5300.0   8.304955   
119        78.30  5300.0   4.385283  ...     6.0  27.0  5300.0   9.907006   
120       121.67  5965.0   7.947695  ...    11.0  47.0  5965.0   7.467006   

In [75]:
speed_pivot_table = avg_travel_time_df.pivot_table(index='time_window',columns='link_id')['speed']

In [42]:
speed_pivot_table.to_json('speed_pivot_table.json',orient='columns')

In [43]:
links

,length,width,lanes,in_top,out_top,lane_width
link_id,,,,,,
100,58,3,1,105,111,3
101,84,3,1,116,121,3
102,131,9,3,115,109,3
103,23,12,4,111,"122,116",3
104,293,9,3,109,112,3
105,78,6,2,NaN,100,3
106,15,3,1,121,113,3
107,34,9,3,123,108,3
108,40,9,3,107,"119,120",3


In [131]:
hours = pd.to_datetime(avg_travel_time_df.time_window).map(lambda d:d.hour)

In [134]:
avg_travel_time_df['hour'] = hours

In [140]:
avg_travel_time_df[['link_id','flow','hour']].groupby(['link_id','hour']).sum()

flow
link_id hour      
100     0      260
        1      177
        2      141
        3      157
        4      194
        5      296
        6      718
        7     1099
        8     1591
        9     1764
        10    1742
        11    1578
        12    1778
        13    1915
        14    2024
        15    1830
        16    1795
        17    1461
        18    1137
        19    1003
        20     911
        21     804
        22     451
        23     343
101     0      133
        1       97
        2       70
        3       58
        4      104
        5      201
...            ...
122     18    2430
        19    2056
        20    1712
        21    1506
        22     955
        23     618
123     0      658
        1      519
        2      465
        3      537
        4      774
        5     1235
        6     2209
        7     3749
        8     5180
        9     5484
        10    4500
        11    3828
        12    4235
        13    4994
        14    4732
        15    4594
        16    4675
        17    4308
        18    3759
        19    3117
        20    2751
        21    2021
        22    1550
        23     897

[576 rows x 1 columns]